In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from scipy import stats
import gmaps

import json

# Import API key
from api_keys import g_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Input File (CSV)
input_data_file = "output_data/cities.csv"


In [2]:
weather_df = pd.read_csv(input_data_file)

weather_df.head()

,Unnamed: 0,Latitude,Longitude,Temperature,Max Temp,Humidity,City,Country,Wind Speed,Cloudiness
0,0,22.0752,-159.3190,82.72,84.11,71,Kapaa,US,8.05,0
1,0,-46.1927,168.8643,51.73,51.73,91,Mataura,NZ,2.35,100
2,0,71.2906,-156.7887,35.62,35.62,93,Barrow,US,20.71,100
3,0,-34.4187,19.2345,54.81,54.81,82,Hermanus,ZA,14.38,59
4,0,50.2536,-96.0612,71.01,71.01,64,Lac du Bonnet,CA,1.90,67


In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,Latitude,Longitude,Temperature,Max Temp,Humidity,City,Country,Wind Speed,Cloudiness
80,0,50.6996,-127.4199,73.58,73.58,64,Port Hardy,CA,8.05,0
110,0,43.7153,45.1264,76.06,76.06,52,Ishcherskaya,RU,5.68,0
163,0,31.4044,30.4164,77.83,77.83,74,Rosetta,EG,9.71,0
172,0,40.4272,71.7189,75.92,75.92,35,Yangi Marg`ilon,UZ,4.61,0
213,0,33.8889,35.4944,71.38,71.38,69,Beirut,LB,4.29,0
227,0,4.8699,-2.2405,72.79,72.79,85,Esim,GH,5.19,0
229,0,-9.0153,-42.6994,75.25,75.25,50,São Raimundo Nonato,BR,6.13,0
241,0,41.1231,80.2644,76.23,76.23,45,Aksu,CN,9.73,0
243,0,42.8000,93.4500,74.34,74.34,39,Kumul,CN,4.18,0
248,0,38.2545,-104.6091,73.74,76.95,51,Pueblo,US,5.75,0


In [9]:
hotel_df = filtered_weather_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?key=" + g_key + "&type=lodging&radius=5000&language=en-us"

for index, row in hotel_df.iterrows():
    # API Parameters    
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # Construct API URL
    url = base_url + "&location={0},{1}".format(lat,lng)
    
    #print("url: " + url)
    
    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.post(url).json()
    
    # extract results
    results = response['results']
    
    #print(results)
    
    try:
        #print(f"Closest hotel is {results[0]['name']}.")
        hotel_name = results[0]['name']
        hotel_zip_city_country = results[0]['plus_code']['compound_code']
        hotel_city = hotel_zip_city_country[1]
        hotel_country = hotel_zip_city_country[2]
        hotel_lat = results[0]['geometry']['location']['lat']
        hotel_lng = results[0]['geometry']['location']['lng']
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    #print("------------")
    hotel_df = hotel_df.dropna(how='any')
    

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [10]:
hotel_df

,Unnamed: 0,Latitude,Longitude,Temperature,Max Temp,Humidity,City,Country,Wind Speed,Cloudiness,Hotel Name
80,0,50.6996,-127.4199,73.58,73.58,64,Port Hardy,CA,8.05,0,Airport Inn-Port Hardy
110,0,43.7153,45.1264,76.06,76.06,52,Ishcherskaya,RU,5.68,0,"МИРАЖ+, усадьба"
213,0,33.8889,35.4944,71.38,71.38,69,Beirut,LB,4.29,0,"Crowne Plaza Beirut, an IHG Hotel"
227,0,4.8699,-2.2405,72.79,72.79,85,Esim,GH,5.19,0,Axim Beach Resort And Spa
229,0,-9.0153,-42.6994,75.25,75.25,50,São Raimundo Nonato,BR,6.13,0,Real Hotel
241,0,41.1231,80.2644,76.23,76.23,45,Aksu,CN,9.73,0,Pudong Holiday Hotel
248,0,38.2545,-104.6091,73.74,76.95,51,Pueblo,US,5.75,0,Courtyard by Marriott Pueblo Downtown
251,0,37.4592,30.5950,73.92,73.92,45,Bucak,TR,6.29,0,Gülay Apart
321,0,40.1948,-92.5833,64.27,71.06,83,Kirksville,US,5.01,0,Budget Host-Village Inn
326,0,8.9242,-67.4293,74.68,74.68,74,Calabozo,VE,3.31,0,Hotel Giardini


In [11]:
# Assign the marker layer to a variable
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))